In [1]:
import torch
from torch import nn,Tensor
from d2l import torch as d2l

In [2]:
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(2,4)
net(X)

tensor([[0.5771],
        [0.5400]], grad_fn=<AddmmBackward0>)

In [5]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.2404,  0.0885, -0.0663,  0.2700,  0.2475,  0.3132,  0.0887,  0.2833]])),
             ('bias', tensor([0.1492]))])

In [6]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1492], requires_grad=True)
tensor([0.1492])


In [7]:
net[2].weight.grad == None

True

In [11]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [12]:
net.state_dict()['2.bias'].data

tensor([0.1492])

In [13]:
def block1():
  return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())
def block2():
  net = nn.Sequential()
  for i in range(4):
    net.add_module(f'block {i}',block1())
  return net
rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[-0.4798],
        [-0.4798]], grad_fn=<AddmmBackward0>)

In [14]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [15]:
rgnet[0][1][0].bias.data

tensor([-0.3519, -0.2398,  0.2460, -0.0285, -0.2049, -0.0919, -0.1101,  0.3108])

In [17]:
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight,mean=0,std=0.01)
    nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([0.0142, 0.0017, 0.0071, 0.0041]), tensor(0.))

In [18]:
def init_constant(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight,1)
    nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]


(tensor([1., 1., 1., 1.]), tensor(0.))

In [19]:
def init_xavier(m):
  if type(m) == nn.Linear:
    nn.init.xavier_normal_(m.weight)
def init_42(m):
  if type(m)==nn.Linear:
    nn.init.constant_(m.weight,42)


net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.0540, -0.1964, -0.5091,  0.2907])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [21]:
# 自定义参数
def my_init(m):
  if type(m) == nn.Linear:
    print("Init",*[(name,param.shape) for name,param in m.named_parameters()][0])
    nn.init.uniform_(m.weight,-10,10)
    m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000, -9.1212,  0.0000],
        [ 0.0000,  5.5796,  0.0000, -0.0000]], grad_fn=<SliceBackward0>)

In [22]:
net[0].weight.data[:] += 1
net[0].weight.data[0,0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -8.1212,  1.0000])

In [23]:
# 共享参数
shared = nn.Linear(8,8)
net  = nn.Sequential(
  nn.Linear(4,8),nn.ReLU(),
  shared,nn.ReLU(),
  shared,nn.ReLU(),
  nn.Linear(8,1)
)
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0]==net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 为什么需要共享参数？
> 共享参数可以节省内存，并且在以下方面具有特定的好处：
> + 对于图像识别中的CNN，共享参数使网络能够在图像中的任何地方而不是仅在某个区域中查找给定的功能。
> + 对于RNN，它在序列的各个时间步之间共享参数，因此可以很好地推广到不同序列长度的示例。
> + 对于自动编码器，编码器和解码器共享参数。 在具有线性激活的单层自动编码器中，共享权重会在权重矩阵的不同隐藏层之间强制正交。